In [43]:
import os

from google.colab import userdata
userdata.get('GIT_TOKEN')
userdata.get('GIT_TOKEN')
userdata.get('OPENAI_TOKEN')

# Store token in env variable (more secure than plain text)
os.environ['GITHUB_TOKEN'] = userdata.get('GIT_TOKEN')

# Use it to clone
!git clone https://$GITHUB_TOKEN@github.com/ShouryaBatra/psbs-research-project.git


fatal: destination path 'psbs-research-project' already exists and is not an empty directory.


In [44]:
!pip install python-dotenv

In [45]:
!pip install -r psbs-research-project/leak_eval/requirements.txt

In [46]:
# gpt env variable

import os
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_TOKEN')

In [47]:
# check gpu

!nvidia-smi

Sun Jul 27 23:03:18 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   40C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [48]:
# clear any cached downloads
!rm -rf ~/.cache/huggingface/hub/models--Qwen--Qwen2.5-7B

In [49]:
# install model

from huggingface_hub import snapshot_download
import os

# Set environment to avoid any caching issues
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"

# Download fresh copy
print("Downloading Qwen2.5-7B model...")
snapshot_download(
    repo_id="Qwen/Qwen2.5-7B",
    local_dir="psbs-research-project/leak_eval/qwen2.5-7b",
    resume_download=False,  # Don't resume corrupted downloads
    local_files_only=False,  # Download from internet
    force_download=True      # Force fresh download
)
print("Download completed!")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Fetching 14 files:   0%|          | 0/14 [00:00<?, ?it/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/686 [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

Download completed!


In [50]:
# copy over prompt folder into leak_eval

!cp -r psbs-research-project/prompts psbs-research-project/leak_eval/

In [51]:
# install airgapagent-r benchmarks

from huggingface_hub import snapshot_download
import os

# Create datasets directory if it doesn't exist
os.makedirs("psbs-research-project/leak_eval/datasets", exist_ok=True)

# Download the airgapagent datasets from Hugging Face
snapshot_download(
    repo_id="parameterlab/leaky_thoughts",
    repo_type="dataset",
    local_dir="./psbs-research-project/leak_eval/datasets",
    ignore_patterns=["*.arrow", "*.lock"]  # Optional: skip unnecessary files
)

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

'/content/psbs-research-project/leak_eval/datasets'

In [52]:
# create airgapagent sample dataset of 3
!python psbs-research-project/leak_eval/scripts/create_sample_dataset.py \
--input_file psbs-research-project/leak_eval/datasets/airgapagent-r-small.json \
--output_file psbs-research-project/leak_eval/datasets/airgapagent-r-sample-3.json \
--sample_size 3 \
--seed 42

Loading data from psbs-research-project/leak_eval/datasets/airgapagent-r-small.json...
Original dataset size: 624
Found 3 unique profiles
Profile distribution:
  Maya Thompson: 208 examples
  Tomoko Nakamura: 208 examples
  Oscar Torres: 208 examples

Sampled 3 examples
Sample distribution:
  Maya Thompson: 1 examples
  Tomoko Nakamura: 1 examples
  Oscar Torres: 1 examples

Saving sampled dataset to psbs-research-project/leak_eval/datasets/airgapagent-r-sample-3.json...
Done! You can now run:
!python eval_cp.py \
--model qwen2.5-1.5b \
--input_file psbs-research-project/leak_eval/datasets/airgapagent-r-sample-3.json \
--output_file results/sample_100_cot.json \
--gpt_eval \
--prompt_type cot_explicit_unk \
--max_tokens 300 \
--temperature 0.1


In [53]:
# create airgapagent sample dataset of 100
!python psbs-research-project/leak_eval/scripts/create_sample_dataset.py \
--input_file psbs-research-project/leak_eval/datasets/airgapagent-r-small.json \
--output_file psbs-research-project/leak_eval/datasets/airgapagent-r-sample-100.json \
--sample_size 100 \
--seed 42

Loading data from psbs-research-project/leak_eval/datasets/airgapagent-r-small.json...
Original dataset size: 624
Found 3 unique profiles
Profile distribution:
  Maya Thompson: 208 examples
  Tomoko Nakamura: 208 examples
  Oscar Torres: 208 examples

Sampled 100 examples
Sample distribution:
  Maya Thompson: 33 examples
  Oscar Torres: 33 examples
  Tomoko Nakamura: 34 examples

Saving sampled dataset to psbs-research-project/leak_eval/datasets/airgapagent-r-sample-100.json...
Done! You can now run:
!python eval_cp.py \
--model qwen2.5-1.5b \
--input_file psbs-research-project/leak_eval/datasets/airgapagent-r-sample-100.json \
--output_file results/sample_100_cot.json \
--gpt_eval \
--prompt_type cot_explicit_unk \
--max_tokens 300 \
--temperature 0.1


In [54]:
# test on airgapagent-r-sample-3 (3 prompts)
# can also change to test on airgapagent-r-sample-100 (don't do this though, that costs a lot of memory)

!cd psbs-research-project/leak_eval && python eval_cp.py \
--model qwen2.5-7b \
--input_file datasets/airgapagent-r-sample-3.json \
--output_file results/test_pipeline_3sample.json \
--gpt_eval \
--prompt_type cot_explicit_unk \
--max_tokens 64 \
--limit 2 \
--temperature 0.1


╭─────────────────────────────────────────────────────────────────────╮
│                         Execution Arguments                         │
│ ╭─────────────────────┬───────────────────────────────────────────╮ │
│ │ Argument            │ Value                                     │ │
│ ├─────────────────────┼───────────────────────────────────────────┤ │
│ │ model               │ qwen2.5-7b                                │ │
│ │ seed                │ 221097                                    │ │
│ │ input_file          │ datasets/airgapagent-r-sample-3.json      │ │
│ │ output_file         │ results/test_pipeline_3sample.json        │ │
│ │ limit               │ 2                                         │ │
│ │ max_tokens          │ 64                                        │ │
│ │ temperature         │ 0.1                                       │ │
│ │ top_p               │ None                                      │ │
│ │ top_k               │ None                                 

In [55]:
# get summary block from new results and print it

import json

file_path = 'psbs-research-project/leak_eval/results/test_pipeline_3sample.json'

with open(file_path, 'r') as f:
    data = json.load(f)

summary_block = data.get('summary')

if summary_block:
    print(json.dumps(summary_block, indent=2))
else:
    print(f"Could not find 'summary' block in {file_path}")

{
  "utility_score": {
    "utility_score_avg": 0,
    "utility_score_std": 0
  },
  "pii_leakage": {
    "output_avg": 0,
    "reasoning_avg": 0,
    "answer_avg": 0,
    "output_bin_avg": 0,
    "reasoning_bin_avg": 0,
    "answer_bin_avg": 0,
    "output_std": 0.0,
    "reasoning_std": 0.0,
    "answer_std": 0.0,
    "output_bin_std": 0.0,
    "reasoning_bin_std": 0.0,
    "answer_bin_std": 0.0
  },
  "total_examples": 2,
  "positive_examples": 0,
  "negative_examples": 2,
  "time_required": "0:02:25",
  "batch_size_used": 4,
  "avg_output_length": 36.5,
  "avg_reasoning_length": 22.5,
  "avg_answer_length": 14.0,
  "avg_close_think_tokens": 1.0,
  "max_close_think_tokens": 1,
  "gpt_utility_score": {
    "gpt_utility_score_avg": 0,
    "gpt_utility_score_std": 0
  },
  "gpt_pii_leakage": {
    "gpt_output_avg": 4.5,
    "gpt_reasoning_avg": 0,
    "gpt_answer_avg": 0,
    "gpt_output_bin_avg": 0.5,
    "gpt_reasoning_bin_avg": 0,
    "gpt_answer_bin_avg": 0,
    "gpt_output_std": 6